In [1]:
import torch
from torch import nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)
filepath = '/content/gdrive/MyDrive' + '/Colab Notebooks/csv/'
mnist = np.load(filepath + 'mnist.npz')

x_train = (mnist['x_train'] - np.mean(mnist['x_train'])) / np.std(mnist['x_train'])
y_train = mnist['y_train']
x_test = (mnist['x_test'] - np.mean(mnist['x_train'])) / np.std(mnist['x_train'])
y_test = mnist['y_test']

print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

Mounted at /content/gdrive
(60000, 28, 28) (60000,) (10000, 28, 28) (10000,)


In [2]:
from torch.utils.data import TensorDataset, DataLoader

# Train Data Set
train_dataset = TensorDataset(torch.FloatTensor(x_train), torch.LongTensor(y_train))
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)

# Test Data Set
test_dataset = TensorDataset(torch.FloatTensor(x_test), torch.LongTensor(y_test))
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [4]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(28*28, 512, bias=False)
        self.fc2 = nn.Linear(512, 512, bias=False)
        self.fc3 = nn.Linear(512, 10, bias=False)

        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(512)

        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.2)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x))) # layer 1
        x = self.dropout1(x)
        x = F.relu(self.bn2(self.fc2(x))) # layer 2
        x = self.dropout2(x)
        x = self.fc3(x) # layer 3

        return x

In [5]:
from torch.optim import SGD

model = MLP().to(device)
criterion  = nn.CrossEntropyLoss().to(device)
opti = SGD(model.parameters(), lr=1e-3)

In [6]:
epoch = 20

model.train() # 모드를 정확히 명시해야, 속도 및 정확성이 유지됨
for i in range(epoch):
    correct = 0
    for data, target in train_dataloader:
        data = data.view(-1, 28*28).to(device)
        target = target.to(device)

        output = model(data)
        loss = criterion(output, target)

        opti.zero_grad()
        loss.backward()
        opti.step()

        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(target.view_as(pred)).sum().item()
    print("%.5f" %(100.0 * correct / len(train_dataloader.dataset)))

76.45500
88.04667
90.02000
91.13833
92.07833
92.67167
93.06167
93.55667
94.10333
94.29833
94.40833
94.87500
95.09500
95.36000
95.38500
95.50833
95.77000
95.92667
95.98000
96.20000


In [7]:
correct = 0

model.eval() # 모델을 검정모드로 변경, 꼭 모델의 모드를 변경 하도록하자!
for data, target in test_dataloader:
    data = data.view(-1, 28*28).to(device)
    target = target.to(device)

    output = model(data)

    pred = output.max(1, keepdim=True)[1]
    correct += pred.eq(target.view_as(pred)).sum().item()
print(100.0 * correct / len(test_dataloader.dataset))

97.32
